In [43]:
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
plt.style.use('ggplot')
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/alex/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
!pwd

/home/alex/Documents/MIDS/w266/final_project/w266_final_project


In [3]:
df1 = pd.read_csv(r'../all-the-news/articles1.csv')
df1.shape

(50000, 10)

In [4]:
df1.head()

,Unnamed: 0,id,title,publication,author,date,year,month,url,content
0,0,17283,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016.0,12.0,NaN,WASHINGTON — Congressional Republicans have...
1,1,17284,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017.0,6.0,NaN,"After the bullet shells get counted, the blood..."
2,2,17285,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,2017.0,1.0,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri..."
3,3,17286,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,2017-04-10,2017.0,4.0,NaN,"Death may be the great equalizer, but it isn’t..."
4,4,17287,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,2017-01-02,2017.0,1.0,NaN,"SEOUL, South Korea — North Korea’s leader, ..."


In [7]:
len(df1.content[0])

5607

In [8]:
df1.content[0]

'WASHINGTON  —   Congressional Republicans have a new fear when it comes to their    health care lawsuit against the Obama administration: They might win. The incoming Trump administration could choose to no longer defend the executive branch against the suit, which challenges the administration’s authority to spend billions of dollars on health insurance subsidies for   and   Americans, handing House Republicans a big victory on    issues. But a sudden loss of the disputed subsidies could conceivably cause the health care program to implode, leaving millions of people without access to health insurance before Republicans have prepared a replacement. That could lead to chaos in the insurance market and spur a political backlash just as Republicans gain full control of the government. To stave off that outcome, Republicans could find themselves in the awkward position of appropriating huge sums to temporarily prop up the Obama health care law, angering conservative voters who have been 

In [41]:
s = 'Donald J. Trump is a piece of shit. He is a psychopathic maniac!'

In [44]:
word_tokenize(s)

['Donal',
 'J.',
 'Trump',
 'is',
 'a',
 'piece',
 'of',
 'shit',
 '.',
 'He',
 'is',
 'a',
 'psychopathic',
 'maniac',
 '!']

In [45]:
sent_tokenize(s)

['Donal J. Trump is a piece of shit.', 'He is a psychopathic maniac!']

In [37]:
pattern = re.compile('[a-z]{2,}\.')
s = 'testing if this. will split J. will split'
re.split(pattern, s.lower())

['testing if ', ' will split j. will split']

In [11]:
?re.split

Signature: re.split(pattern, string, maxsplit=0, flags=0)
Docstring:
Split the source string by the occurrences of the pattern,
returning a list containing the resulting substrings.  If
capturing parentheses are used in pattern, then the text of all
groups in the pattern are also returned as part of the resulting
list.  If maxsplit is nonzero, at most maxsplit splits occur,
and the remainder of the string is returned as the final element
of the list.
File:      ~/anaconda3/lib/python3.7/re.py
Type:      function


# define biased lexicons and bigrams

In [5]:
lib_bigrams = [
    'thinkprogress org',
    'video thinkprogress',
    'et rally',
    'org 2008',
    'gi bill',
    'wonk room',
    'dana perino',
    'phil gramm',
    'senator mccain',
    'abu ghraib'
]

con_bigrams = [
    'exit question',
    'hat tip',
    'ed lasky',
    'hot air',
    'tony rezko',
    'ed morrissey',
    'track record',
    'confirmed dead',
    'american thinker',
    'illegal alien',
]